In [ ]:
# Import packages and read-in files

import numpy as np
import pandas as pd
import xpress as xp
from datetime import datetime, timedelta
import os

# Read in files using the explicitly defined base path
ch_0_conversion_rates = pd.read_csv('channel_0_conversion_rates.csv')
ch_0_schedule = pd.read_csv('channel_0_schedule.csv')
ch_1_conversion_rates = pd.read_csv('channel_1_conversion_rates.csv')
ch_1_schedule = pd.read_csv('channel_1_schedule.csv')
ch_2_conversion_rates = pd.read_csv('channel_2_conversion_rates.csv')
ch_2_schedule = pd.read_csv('channel_2_schedule.csv')
ch_A_schedule = pd.read_csv('channel_A_schedule.csv')
movies_df = pd.read_csv('movie_database.csv')

In [ ]:
# To use right xpress and get rid of unnecessary error codes
xp.init('C:/xpressmp/bin/xpauth.xpr')
pd.options.mode.copy_on_write = True

In [ ]:
# FORMATING
# Convert 'Date-Time' columns to datetime format
date_cols = ['Date']

for df in [ch_0_conversion_rates, ch_0_schedule, ch_1_conversion_rates, ch_1_schedule,
           ch_2_conversion_rates, ch_2_schedule, ch_A_schedule]:
    df['Date'] = pd.to_datetime(df['Unnamed: 0'])
    df.set_index('Date', inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace = True)
   

# Convert 'Release Date' in movie_database to datetime
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

# Fill missing values if necessary
movies_df.fillna(0, inplace=True)


In [ ]:
# Make 30-min slots
# Slot duration 30 minutes
slot_duration = 30  # minutes
slots_needed = (movies_df['runtime_with_ads'] / slot_duration).apply(lambda x: int(x)).astype(int)
movies_df['slots_needed'] = slots_needed

In [ ]:
# Check for duplicate movie titles
duplicate_titles = movies_df[movies_df.duplicated(subset=['title'], keep=False)]
if not duplicate_titles.empty:
    print("Duplicate movie titles found:")
    print(duplicate_titles['title'])
else:
    print("No duplicate movie titles found.")

In [ ]:
# Make broadcast date for any number of days

# Choose the number of days you're make a schedule for 
    # this can  be used in the functions later
num_days = 7

# Define the broadcasting start and end dates
broadcast_start_date = datetime.strptime("2024-10-01", "%Y-%m-%d")
broadcast_end_date = broadcast_start_date + timedelta(days=num_days)  # 7 days including start date

# Define daily broadcast start and end times
daily_broadcast_start_time = timedelta(hours=7, minutes=0)
daily_broadcast_end_time = timedelta(hours=23, minutes=30)

# Generate all time slots over the date range
time_slots = []
current_date = broadcast_start_date
while current_date <= broadcast_end_date:
    # Set the start and end times for the current day
    day_start = datetime.combine(current_date.date(), datetime.min.time()) + daily_broadcast_start_time
    day_end = datetime.combine(current_date.date(), datetime.min.time()) + daily_broadcast_end_time

    current_time = day_start
    while current_time <= day_end:
        time_slots.append(current_time)
        current_time += timedelta(minutes=slot_duration)

    # Move to the next day
    current_date += timedelta(days=1)

# Create mappings between time slots and indices
time_to_index = {t: idx for idx, t in enumerate(time_slots)}
index_to_time = {idx: t for idx, t in enumerate(time_slots)}

In [ ]:
# Make a smaller movies dataframe so it doesn't take so long
movies_small = movies_df.head(300)
movies_small.set_index('title', inplace = True)

In [ ]:
# Scheduling Optimizer Model
def model(T, movies):
    prob = xp.problem(name="Movie_Scheduling_Problem")
    # M = range(len(movies_small))
    # Decision Variables
    # if movie m in shown in time slot t
    x = {(m,t): xp.var(vartype=xp.binary, name='x{0}_{1}'.format(m,t)) 
         for t in T for m in movies.index}
    prob.addVariable(x)
    
    y = {(m): xp.var(vartype=xp.binary, name='y{0}'.format(m)) for m in movies.index}
    prob.addVariable(y)

    # start time of movie m 
    s = {(m): xp.var(vartype=xp.integer, name='s{0}'.format(m)) for m in movies.index}
    prob.addVariable(s)

    # end time of movie m 
    e = {(m): xp.var(vartype=xp.integer, name='e{0}'.format(m)) for m in movies.index}
    prob.addVariable(e)
    decision_vars = [x,y,s,e]
    

    # Constraints
    # for t in T:
        # big M 
    M = len(time_slots)
    T_end = len(time_slots) -1 # last time slot index
    
    # movie duration 
    prob.addConstraint(xp.Sum(x[m,t] for t in T) == movies.loc[m, 'slots_needed']*y[m] for m in movies.index)

    # has to be a movie in every slot
    prob.addConstraint(xp.Sum(x[m,t] for m in movies.index) == 1 for t in T)

    # end time
    # prob.addConstraint(e[m] == t*xp.Sum(x) for m in movie.index())

    # end time limit
    prob.addConstraint((t+1)*x[m,t] <= e[m] for m in movies.index for t in T)

    # start time limit
    #prob.addConstraint(s[m] <= (t*x[m,t]) for m in movie.index() fot t in T)

    # latest time a movie can start and still show full movie 
    prob.addConstraint(s[m] <= t*x[m,t] + (1-x[m,t])*M for m in movies.index for t in T)

    # end-start = movie duration
    prob.addConstraint(e[m]-s[m] == movies.loc[m, 'slots_needed']*y[m] for m in movies.index)

    # last movie time?
    prob.addConstraint(s[m] + movies.loc[m, 'slots_needed'] -1 <= T_end for m in movies.index)

    return prob, decision_vars

In [ ]:
# Making schedule layout
def get_time(slot_index):
    return time_slots[slot_index].time()

used_movie_ids = []
def get_sched(prob, movies, decision_vars): 
    
    x = decision_vars[0]
    y = decision_vars[1]
    s = decision_vars[2]
    e = decision_vars[3]
    
    scheduled_movies = []
    used_movie_ids = []

    for m in movies_small.index:
    # Retrieve the solution value for y[m_idx]
        y_value = prob.getSolution(y[m])
        
        if y_value > 0.5:  # Movie is scheduled
            used_movie_ids.append(m)
            
            # # Retrieve solution values for s[m_idx] and e[m_idx]
            start_slot = int(prob.getSolution(s[m]))
            end_slot = int(prob.getSolution(e[m]))
            
            # # # # Convert slot indices to actual times
            start_time = get_time(start_slot)
            end_time = get_time(end_slot)
    
            # # Append the scheduled movie details
            scheduled_movies.append({
                'Movie Title': m,
                'Start Slot': start_slot,
                'Start Time': start_time.strftime('%H:%M'),
                'End Slot': end_slot,
                'End Time': end_time.strftime('%H:%M')
            })
    schedule_df = pd.DataFrame.from_dict(scheduled_movies)
    schedule_df.sort_values(['Start Slot'], axis = 0, inplace = True)
    print(schedule_df)
    return scheduled_movies, used_movie_ids
    

In [ ]:
# Putting it all together

